In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import datetime
# from pykovy.src.pykovy import chain
# import networkx as nx
import matplotlib.pyplot as plt
from time import time

import math

import tensorflow as tf

from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Flatten, Reshape, Input, BatchNormalization, Bidirectional
from tensorflow.python.keras.optimizers import Adam
# from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.layers import add, concatenate
from tensorflow.python.keras.callbacks import ModelCheckpoint

print('after import')

# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

In [5]:
df = pd.read_csv('./all_data.csv')
df = df[['Client Username', 'Association Time', 'AP Name', 'Session Duration', 'Client MAC Address']]
df = df.assign(id=(df['Client Username'] + '_' + df['Client MAC Address']).astype('category').cat.codes)


In [6]:
df = df.drop(['Client Username', 'Client MAC Address'], axis=1)
df.rename(columns={'id':'Client Username'}, inplace=True)
df = df[['Client Username', 'Association Time', 'AP Name', 'Session Duration']]


In [7]:
# recode map location
ap_dict = {}
count = 0
for i in df['AP Name'].unique():
    ap_dict[i] = count
    count += 1
df['AP Name'] = df['AP Name'].map(ap_dict)

In [9]:
def timer(func):
    # this part is for timer ignore
    def wrapper(*arg):
        before = time()
        rv = func(*arg)
        after = time()
        print('Elapsed: ', after-before)
        return rv
    return wrapper

def convertMonth(month):
    if month == 'Jan': return 1
    elif month == 'Feb': return 2
    elif month == 'Mar': return 3
    elif month == 'Apr': return 4
    elif month == 'May': return 5
    elif month == 'Jun': return 6
    elif month == 'Jul': return 7
    elif month == 'Aug': return 8
    elif month == 'Sep': return 9
    elif month == 'Oct': return 10
    elif month == 'Nov': return 11
    elif month == 'Dec': return 12
    else: print('error input is not correct')
        
def convert_to_timeslot(duration):
    temp = duration.split()
    if len(temp) == 2: sess_duration = int(temp[0])
    elif len(temp) == 4: sess_duration = int(temp[0]) * 60 + int(temp[2])
    elif len(temp) == 6: sess_duration = (int(temp[0]) * 60 + int(temp[2])) * 60 + int(temp[4])
    else: print('error')
    return datetime.timedelta(seconds=sess_duration)

def convert_to_datetime(time):
    temp = time.split()
    d_time = datetime.datetime(year=int(temp[5]), month=convertMonth(temp[1]), day=int(temp[2]), hour=int(temp[3].split(':')[0]), minute=int(temp[3].split(':')[1]), second=int(temp[3].split(':')[2]))
    return d_time

# data_list has same number of row as df.
# Each row [userid, map_loc, associate_time, associate_time+sess_dur, sess_dur].
def create_data_list(df):
    np_df = df.values
    data_list = []
    c=0
    for i in range(np_df.shape[0]):
        if c%500000 == 0:
            print(c)
        temp = np_df[i]
#         sess = convert_to_timeslot(df.iloc[i,3])
#         s_time = convert_to_datetime(df.iloc[i,1])
#         e_time = s_time + sess
#         data_list.append([df.iloc[i,0], df.iloc[i,2], s_time, e_time, sess])
        sess = convert_to_timeslot(temp[3])
        s_time = convert_to_datetime(temp[1])
        e_time = s_time + sess
        data_list.append([temp[0], temp[2], s_time, e_time, sess])
        c+=1
        
    data_list = sorted(data_list, key=lambda element: (element[0], element[2]))
    data_list1 = sorted(data_list, key=lambda element: (element[2]))
    first = data_list1[0][2]
    last = data_list1[-1][2]
    return data_list, first, last

# add up the session duration when the previous row has the same userid and location. 
# (this happen when you disconnect from the ap and reconnect again or your wifi session expire, etc.)
def preprocess_data_list(data_list):
    for i in reversed(range(1, len(data_list))):
        ts_cur = data_list[i][2]
        ts_prev = data_list[i-1][2]
        if data_list[i][0] == data_list[i-1][0] and data_list[i][1] == data_list[i-1][1] and ts_cur.day == ts_prev.day and ts_cur.month == ts_prev.month:
#             if data_list[i][2] - data_list[i-1][3] <= datetime.timedelta(hours=2):
            data_list[i-1][4] += data_list[i][4]
#             else: print(i)
    # loop each row, remove the next concecutive rows has the same userid and map_loc with the current row.
    # Because they are useless now
    final_data_list = []
    i = 0
    while i < len(data_list)-1:
        ts_cur = data_list[i][2]
        ts_next = data_list[i+1][2]
        if data_list[i][0] == data_list[i+1][0] and data_list[i][1] == data_list[i+1][1] and ts_cur.day == ts_next.day and ts_cur.month == ts_next.month:
            final_data_list.append(data_list[i])
            c = 2
            while i+c < len(data_list) and data_list[i][0] == data_list[i+c][0] and data_list[i][1] == data_list[i+c][1] and ts_cur.day == ts_next.day and ts_cur.month == ts_next.month:
                c += 1
            i += c

        else:
            final_data_list.append(data_list[i])
            i += 1

    if not (final_data_list[-1][0] == data_list[-1][0] and final_data_list[-1][1] == data_list[-1][1]):
        final_data_list.append(data_list[-1])

    return final_data_list


# user_dict's key are each userid and value is a list of all rows of each user in final_data_list
def create_user_dict(final_data_list):
    user_dict = defaultdict(list)
    for row in final_data_list:
        user_dict[row[0]].append([row[2], row[1], row[4]])

    return user_dict

# 
def remove_jitter(final_data_list):
    res = [final_data_list[0]]
    i = 1
    mem = 0
    while i < len(final_data_list):
        cur_usrid = final_data_list[i][0]
        cur_loc = final_data_list[i][1]
        cur_sess = final_data_list[i][4]
        cur_end_time = final_data_list[i][2] + final_data_list[i][4]
        cont_time = final_data_list[i][2] - (res[-1][2] + res[-1][4]) <= datetime.timedelta(minutes=1)
        if cur_usrid == res[-1][0] and cur_sess <= datetime.timedelta(minutes=1) and cont_time:


            if cur_loc != res[-1][1] and mem == 0:
                mem = i

            elif cur_loc == res[-1][1]:
                
#                 print(i, cur_loc, cur_end_time, res[-1][1], res[-1][2])

#                 print(cur_end_time - res[-1][2])
                res[-1][4] = cur_end_time - res[-1][2]
#                 print(res[-1][4])
                mem=0
                
            else: pass
        elif cur_usrid == res[-1][0] and cur_sess <= datetime.timedelta(minutes=1) and not cont_time:
            if cur_loc != res[-1][1] and mem == 0:
                res.append(final_data_list[i])

            elif cur_loc == res[-1][1]:
                res[-1][4] = cur_end_time - res[-1][2]
                mem=0
                
            else:                 
                for j in range(mem, i):
                    res.append(final_data_list[j])
                res.append(final_data_list[i])
                mem=0
        else:
            if mem != 0:
                for j in range(mem, i):
                    res.append(final_data_list[j])
                res.append(final_data_list[i])
                mem=0
            else: res.append(final_data_list[i])
        
        i+=1

    return res

@timer
def preprocess_and_create_user_dict(df):
    data_list, start_ts, end_ts= create_data_list(df)
    final_data_list = preprocess_data_list(data_list)
    print(len(final_data_list), len(data_list))
    final_data_list = remove_jitter(final_data_list)
    print(len(final_data_list), len(data_list))
    user_dict = create_user_dict(final_data_list)
    return user_dict, start_ts, end_ts


In [10]:
user_dict, start_ts, end_ts = preprocess_and_create_user_dict(df)


0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
4061485 6114987
4061202 6114987
Elapsed:  97.63698029518127


In [13]:
a = [i for i in user_dict if len(user_dict[i]) < 20]

In [16]:
t = map(user_dict.pop, a)
for i in range(len(a)):
    try:
        next(t)
    except:
        print('fail')

In [22]:
def convert_to_timeslot1(d_time):
    slot = (d_time.hour * 60 + d_time.minute)/5
    return int(slot) if slot < int(slot) + 0.5 else int(slot) + 1

def create_data(user_dict):
    input_data = []
    c = 0
    for usr in user_dict:
        c+=1
        if c % 5000 == 0: print(c)
        for j in range(len(user_dict[usr] )-1):
            loc = user_dict[usr][j][1]
            input_data.append([usr, user_dict[usr][j][0].weekday(), loc])
            
    return input_data


In [23]:
input_data = create_data(user_dict)

5000
10000
15000
20000
25000
30000


In [24]:
df1 = pd.DataFrame(data=input_data, columns=['userID', 'weekday', 'location'])


In [26]:
map_dict_time_loc = {}
num_loc_time = 1
for i in df1['location'].unique():
    map_dict_time_loc[i] = num_loc_time
    num_loc_time += 1
df1['location'] = df1['location'].map(map_dict_time_loc)
np_df1 = df1.values

In [27]:
input_dict = defaultdict(list)
week_dict = defaultdict(list)
for i in range(np_df1.shape[0]):
    if i % 500000 == 0: print(i)
    temp = np_df1[i]
    input_dict[temp[0]].append(temp[2])
    week_dict[temp[0]].append(temp[1])
    

0
500000
1000000
1500000
2000000
2500000
3000000
3500000


In [28]:
X_data = []
y_data = []
usr_data = []
week_data = []
window = 8

for usr in input_dict:
#     if usr != 29: continue 
    size = len(input_dict[usr])
    
    if size == 0: continue
    elif size <= window:
        temp = []
        for _ in range(window - size):
            temp.append(0)
        for i in range(size-1):
            temp.append(input_dict[usr][i])
        week_data.append(week_dict[usr][-2])
        X_data.append(temp)
        usr_data.append(usr)
        try:
            y_data.append(input_dict[usr][-1])
        except:
            print(usr)
    else:
        for i in range(size - (window + 1)):
            temp=[]
            for j in range(window+1):
                if j == window:
                    y_data.append(input_dict[usr][i+j])
                elif j == window - 1:
                    temp.append(input_dict[usr][i+j])
                    week_data.append(week_dict[usr][i+j])
                else:
                    temp.append(input_dict[usr][i+j])
 
            X_data.append(temp)
            usr_data.append(usr)

    

In [29]:
X_data = np.array(X_data)
y_data = np.array(y_data)
usr_data = np.array(usr_data)
usr_data1 = np.zeros(len(usr_data))
week_data = np.array(week_data)
# reid user
temp = {}
count = -1
for i in range(len(usr_data)):
    if usr_data[i] not in temp:
        count += 1
        temp[usr_data[i]] = count
        usr_data1[i] = count
    else:
        usr_data1[i] = temp[usr_data[i]]



In [30]:
a = len(set(usr_data))
usr_data = usr_data1.reshape(usr_data1.shape[0], 1)
week_data = week_data.reshape(week_data.shape[0], 1)

In [ ]:
embedding_size = 50

In [ ]:
modela = Sequential()
modela.add(Embedding(input_dim=num_loc_time,
                    output_dim=embedding_size,
                    input_length=window,
                    name='layer_embedding'))

# modela.add(Bidirectional(LSTM(128, return_sequences=True)))
modela.add(Bidirectional(LSTM(64, return_sequences=True)))
modela.add(Bidirectional(LSTM(32)))

# modela.add(Dense(500, activation='relu'))
modela.add(Dense(200, activation='relu'))


modelb = Sequential()
modelb.add(Embedding(input_dim=a,
                    output_dim=20,
                    input_length=1,
                    name='layer_embedding1'))

modelb.add(Dense(100, activation='relu'))
modelb.add(Flatten())

modelc = Sequential()
modelc.add(Dense(5, activation='relu', input_shape=(7,)))
modelc.add(Flatten())


merged_output = concatenate([modela.output, modelb.output, modelc.output])
model_combined = Sequential()

model_combined.add(Dense(num_loc_time, activation='softmax'))

final_model = Model([modela.input, modelb.input, modelc.input], model_combined(merged_output))
final_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])


In [ ]:
y_data1 = to_categorical(y_data)
week_data1 = to_categorical(week_data)

In [ ]:
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [ ]:
final_model.fit([X_data, usr_data, week_data1], y_data1, validation_split=0.1, epochs=20, batch_size=256, callbacks=callbacks_list, verbose=0)


/home/011816337/.local/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2957725 samples, validate on 328637 samples
Epoch 1/20
2957725/2957725 [==============================] - 941s 318us/step - loss: 4.0132 - acc: 0.2280 - val_loss: 4.1209 - val_acc: 0.1943
Epoch 2/20
2957725/2957725 [==============================] - 935s 316us/step - loss: 3.7364 - acc: 0.2700 - val_loss: 4.0896 - val_acc: 0.2044
Epoch 3/20
 473344/2957725 [===>..........................] - ETA: 12:35 - loss: 3.6922 - acc: 0.2752